In [1]:
import Levenshtein
import wikitextparser as wtp
import os
import sys
sys.path.append('../')

from wiki_dump_reader import iterate
from tqdm import tqdm

from external_sources_module import *
from Identification_couples_livres.extract_books_from_DB import *

if os.path.isfile('../Data/all_books.json'):
    with open('../Data/all_books.json', 'r') as input_file:
        all_books = json.load(input_file, cls=BookJSONDecoder)
else:
    all_books = generate_all_books()

if os.path.isfile('../Data/author_ls.json'):
    with open('../Data/author_ls.json', 'r') as input_file:
        author_ls = json.load(input_file, cls=BookJSONDecoder)
else:
    author_ls, raw_author_ls = generate_all_authors(all_books)

# Wikipédia

## Extraction des pages Wikipédia

### Parcours du dump et présélection des pages
On sélectionne parmis le dump wikipédia les pages qui concernent des livres ou des auteurs

In [5]:
book_to_keep = {}
writer_to_keep = {}
count_author = 0
count_book = 0

for title, text in tqdm(iterate('../Data/Wikipedia/frwiki-20200401-pages-articles-multistream.xml'), total=20200401):

    is_book_re = re.search(r"Infobox (Ouvrage|Livre)", text)
    is_ebauche_re = re.search(r"{{Ébauche\|livre}}", text)
    if is_book_re or is_ebauche_re:
        book_to_keep[title] = text
        count_book += 1

    is_writer_re = re.search(r"Infobox Écrivain", text)
    is_Ecrivain_Quebecois_re = re.search(r"Catégorie:Écrivain", text)
    if is_writer_re or is_Ecrivain_Quebecois_re:
        writer_to_keep[title] = text
        count_author += 1

print('Nombre de page de livres extraits: ', count_book)
with open('../Data/Wikipedia/fr_dumps_wikipedia_books.json', 'w') as outfile:
    json.dump(book_to_keep, outfile)
print('Nombre de page d\'auteurs extraits: ', count_author)
with open('../Data/Wikipedia/fr_dumps_wikipedia_writers.json', 'w') as outfile:
    json.dump(writer_to_keep, outfile)

 19%|█▉        | 3799564/20200401 [16:15<1:10:12, 3893.52it/s]


count_books:  16749
count_authors:  40625


### Comparaison avec la base de donnée
On recherche maintenant les auteurs et livres en communs avec la base de donnée

In [6]:
# Couple dont on estime que les deux auteurs sont les mêmes, en fonction du critère (ex: 4 => 1/4)
couple_books_4 = []
couple_books_6 = []
couple_books_8 = []
# Compte les livres des auteurs par base de donnée (permet de voir si une base de donnée n'a aucun match)
data_base_stats_4 = {}
data_base_stats_6 = {}
data_base_stats_8 = {}

# Compte les auteurs de wikipédia qui ont été trouvé dans nos bases de donnée
found_at_least_once = 0
# Compte le nombre de couples
nb_couple = 0
# Compte le nombre de fois qu'un auteur de wikidia été trouvé dans nos bases de donnée
author_found_freq = {}

with open('../Data/Wikipedia/fr_dumps_wikipedia_books.json', 'r') as input_file:
    wiki_books = json.load(input_file)

for wiki_book, wiki_text in tqdm(wiki_books.items()):
    # Compte le nombre de fois que l'auteur wikipédia a été associé a un auteur de nos bases de donnée
    wiki_author_found_count = 0
    normalized_wiki_book = normalize(wiki_book)
    for DB_book in all_books:
        dist_auteur = Levenshtein.distance(DB_book.title, normalized_wiki_book)
        if dist_auteur < max(1, min(len(DB_book.title), len(normalized_wiki_book)) / 4):
            new_couple = {
                        'book DB': DB_book,
                        'book wiki': {'title': normalized_wiki_book, 'title_raw':wiki_book, 'text': wiki_text}
                    }
            nb_couple += 1
            wiki_author_found_count += 1

            if dist_auteur < max(1, min(len(DB_book.title), len(wiki_book)) / 8):
                    couple_books_8.append(new_couple)
                    try:
                        data_base_stats_8[DB_book.data_base] += 1
                    except:
                        data_base_stats_8[DB_book.data_base] = 1
            elif dist_auteur < max(1, min(len(DB_book.title), len(wiki_book)) / 6):
                couple_books_6.append(new_couple)
                try:
                    data_base_stats_6[DB_book.data_base] += 1
                except:
                    data_base_stats_6[DB_book.data_base] = 1
            else:
                couple_books_4.append(new_couple)
                try:
                    data_base_stats_4[DB_book.data_base] += 1
                except:
                    data_base_stats_4[DB_book.data_base] = 1

    if wiki_author_found_count:
        found_at_least_once += 1
        author_found_freq[wiki_book] = wiki_author_found_count

print("Nombre d'autheurs en commun: ", found_at_least_once, " soit ", found_at_least_once * 100 / len(wiki_books), "%")
print("Nombre d'autheurs reconnus: ", nb_couple, " soit ", nb_couple/found_at_least_once, ' livre dans nos bases de donnée par livre reconnu dans wikipédia')
print("\nFrequence de reconnaissance des 10 auteurs de wikipédia les plus reconnus: ", sorted(author_found_freq.items(), key=lambda item: item[1], reverse=True)[:10])

print("\nRépartition des livres des auteurs reconnus dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats_8))
print('1/6: ', json.dumps(data_base_stats_6))
print('1/4: ', json.dumps(data_base_stats_4))

with open('../Data/Wikipedia/couple_books_wikipedia_4.json', 'w') as outfile:
    json.dump(couple_books_4, outfile, cls=BookJSONEncoder)

with open('../Data/Wikipedia/couple_books_wikipedia_6.json', 'w') as outfile:
    json.dump(couple_books_6, outfile, cls=BookJSONEncoder)

with open('../Data/Wikipedia/couple_books_wikipedia_8.json', 'w') as outfile:
    json.dump(couple_books_8, outfile, cls=BookJSONEncoder)

100%|██████████| 16749/16749 [36:43<00:00,  7.60it/s]


nombre d'autheurs en commun:  3853  soit  23.004358469162337 %
nombre d'autheurs reconnus:  13116  soit  3.4041007007526605  livre dans nos bases de donnée par livre reconnu dans wikipédia
Frequence de reconnaissance:  [('Maria Chapdelaine', 92), ('Le Petit Chaperon rouge', 38), ('Les Anciens Canadiens', 38), ("Les Chevaliers d'Émeraude", 35), ('Poems (Agatha Christie)', 35), ('Un homme et son péché (roman)', 34), ('La Reine des anges', 31), ('La Reine des neiges (roman)', 30), ('La Reine des neiges', 30), ('Agaguk', 28)]
repartition dans les bases de données: 
1/8:  {"Depot_legal": 2852, "ADP": 313, "ILE": 1396, "Babelio": 106}
1/6:  {"Depot_legal": 700, "ILE": 320, "ADP": 83, "Babelio": 14}
1/4:  {"Depot_legal": 4661, "ILE": 2028, "Babelio": 145, "ADP": 498}


In [ ]:
# Couple dont on estime que les deux auteurs sont les mêmes, en fonction du critère (ex: 4 => 1/4)
couple_writers_4 = []
couple_writers_6 = []
couple_writers_8 = []

# Compte les livres des auteurs par base de donnée (permet de voir si une base de donnée n'a aucun match)
data_base_stats_4 = {}
data_base_stats_6 = {}
data_base_stats_8 = {}

# Compte les auteurs de wikipédia qui ont été trouvé dans nos bases de donnée
found_at_least_once = 0
# Compte le nombre de couples
nb_couple = 0
# Compte le nombre de fois qu'un auteur de wikidia été trouvé dans nos bases de donnée
author_found_freq = {}

with open('../Wikipedia/fr_dumps_wikipedia_writers.json', 'r') as input_file:
    wiki_authors = json.load(input_file)

for wiki_author, wiki_author_text in tqdm(wiki_authors.items()):
    # Compte le nombre de fois que l'auteur wikipédia a été associé a un auteur de nos bases de donnée
    wiki_author_found_count = 0
    normalized_wiki_author = normalize_author(wiki_author)
    for DB_author, DB_author_books in author_ls.items():
        dist_auteur = Levenshtein.distance(DB_author, normalized_wiki_author)

        if dist_auteur < max(1, min(len(DB_author), len(normalized_wiki_author)) / 4):
            new_couple = {
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_wiki': {'name': normalized_wiki_author, 'name_raw': wiki_author, 'text':wiki_author_text}
                    }

            nb_couple += 1
            wiki_author_found_count += 1

            if dist_auteur < max(1, min(len(DB_author), len(normalized_wiki_author)) / 8):
                    couple_writers_8.append(new_couple)
                    for book in DB_author_books:
                        try:
                            data_base_stats_8[book.data_base] += 1
                        except:
                            data_base_stats_8[book.data_base] = 1
            elif dist_auteur < max(1, min(len(DB_author), len(normalized_wiki_author)) / 6):
                couple_writers_6.append(new_couple)
                for book in DB_author_books:
                    try:
                        data_base_stats_6[book.data_base] += 1
                    except:
                        data_base_stats_6[book.data_base] = 1
            else:
                couple_writers_4.append(new_couple)
                for book in DB_author_books:
                    try:
                        data_base_stats_4[book.data_base] += 1
                    except:
                        data_base_stats_4[book.data_base] = 1

    if wiki_author_found_count:
        found_at_least_once += 1
        author_found_freq[wiki_author] = wiki_author_found_count
    # if count_freq_i > 10:
    #     print("compte superieur a 10: ", author_DB)

print("Nombre d'autheurs en commun: ", found_at_least_once, " soit ", found_at_least_once * 100 / len(wiki_authors), "%")
print("Nombre d'autheurs reconnus: ", nb_couple, " soit ", nb_couple/found_at_least_once, ' auteur dans nos bases de donnée par auteur reconnu dans wikipedia')
print("\nFrequence de reconnaissance des 10 auteurs de wikipédia les plus reconnus: ", sorted(author_found_freq.items(), key=lambda item: item[1], reverse=True)[:10])


print("\nRépartition des livres des auteurs reconnus dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats_8))
print('1/6: ', json.dumps(data_base_stats_6))
print('1/4: ', json.dumps(data_base_stats_4))

with open('../Data/Wikipedia/couple_authors_wikipedia_4.json', 'w') as outfile:
    json.dump(couple_writers_4, outfile, cls=BookJSONEncoder)

with open('../Data/Wikipedia/couple_authors_wikipedia_6.json', 'w') as outfile:
    json.dump(couple_writers_6, outfile, cls=BookJSONEncoder)

with open('../Data/Wikipedia/couple_authors_wikipedia_8.json', 'w') as outfile:
    json.dump(couple_writers_8, outfile, cls=BookJSONEncoder)

### Parsing des informations de la page wikipédia des couples extraits

In [8]:
with open('../Data/Wikipedia/couple_authors_wikipedia_8.json', 'r') as outfile:
    couple_writers = json.load(outfile, cls=BookJSONDecoder)

# On compte les intitulés des sections pour la suite
title_dict = {}

for couple in tqdm(couple_writers):

    info_box, page = split_info_box(couple['author_wiki']['text'])

    couple['author_wiki']['info_box'] = get_info_from_infobox(info_box)

    parsed_page = wtp.parse(format_list(page))
    sections_infos = {}
    section_titles = set()

    for section in parsed_page.sections:
        if section.level <= 2:
            section_titles.add(section.title)
            sections_infos[section.title] = parse_section(section, section_titles, section.level)


    couple['author_wiki']['parsed_text'] = sections_infos

    # On met a jour le compte des intitulés des sections
    for title in section_titles:
        try:
            title_dict[title] += 1
        except KeyError:
            title_dict[title] = 1


print("Titre des sections les plus fréquentes: ")
for name, nb in dict(sorted(title_dict.items(), key=lambda item: item[1], reverse=True)[:15]).items():
    print('Section: ', name + " " * (30 - len(name)),  ' compte: ', nb)

with open('../Data/Wikipedia/parsed_couple_writers.json', 'w') as outfile:
    json.dump(couple_writers, outfile, cls=BookJSONEncoder)

100%|██████████| 2412/2412 [00:10<00:00, 231.30it/s]


titre des sections:  [
  [
    null,
    2412
  ],
  [
    " Liens externes ",
    1977
  ],
  [
    " Biographie ",
    1720
  ],
  [
    " Notes et r\u00e9f\u00e9rences ",
    1423
  ],
  [
    " \u0152uvres ",
    810
  ],
  [
    " Bibliographie ",
    706
  ],
  [
    " Romans ",
    521
  ],
  [
    " \u0152uvre ",
    458
  ],
  [
    " Voir aussi ",
    417
  ],
  [
    " R\u00e9f\u00e9rences ",
    409
  ],
  [
    " Honneurs ",
    298
  ],
  [
    " Articles connexes ",
    291
  ],
  [
    " Publications ",
    223
  ],
  [
    " Th\u00e9\u00e2tre ",
    211
  ],
  [
    " Annexes ",
    208
  ],
  [
    " Po\u00e9sie ",
    189
  ],
  [
    " Nouvelles ",
    179
  ],
  [
    " Prix et distinctions ",
    176
  ],
  [
    " Filmographie ",
    165
  ],
  [
    " Essais ",
    152
  ],
  [
    " Sources ",
    147
  ],
  [
    " Distinctions ",
    141
  ],
  [
    " Notes ",
    124
  ],
  [
    "Biographie",
    116
  ],
  [
    " Recueils de nouvelles ",
    106
  ]
]


## Confirmation des les couples par leurs informations
On s'intéresse aux information récoltées sur ces auteurs

In [ ]:
with open('../Data/Wikipedia/parsed_couple_writers.json', 'r') as input_file:
    couples_writers = json.load(input_file, cls=BookJSONDecoder)

count_awards = 0
nb_awards_for_each_wiki_author = []

all_awards_name = set()

for couple in couples_writers:
    award_titles = set()

    for title, section in couple['author_wiki']['parsed_text'].items():
        if re.search(r'prix|distinction', title, re.IGNORECASE):
            awards = extract_title_from_list(section['list'])
            award_titles.update(awards)

    if award_titles:
        count_awards += 1
        all_awards_name.update(award_titles)
    nb_awards_for_each_wiki_author.append(len(award_titles))
print("Nombre d'auteur dont on a récupéré des prix: ", count_awards)
print("Nombre total de prix: ", sum(nb_awards_for_each_wiki_author))

print("exemple des prix: ", all_awards_name[:20])

On veut confirmer nos couples d'auteur a partir des titres des livres qu'ils ont ecrits
On va comparer les titres uns à uns.
- Si un des titre ou plus correspond, alors on confirmera ce couple,
- Si aucune titre n'est en commun, on considera le couple comme faux ou incomplet
- Si on n'a pas récupéré de titre depuis wikidata alors on ne peut pas conclure

In [ ]:
with open('../Data/Wikipedia/parsed_couple_writers.json', 'r') as couples_file:
    couple_writers = json.load(couples_file, cls=BookJSONDecoder)

# Listes pour stoker nos couples confirmé, invalidé et ceux dont on ne peut pas conclure car il n'y a pas de titres
couples_confirmed = []
couples_infirmed = []
couples_unsure = []
# Compte les cas ci-dessus
count_checked_by_titles = 0
count_differentiate_by_titles = 0
count_no_titles = 0

for couple in tqdm(couple_writers):
    confirmed_by_title_match = False
    confirmed_by_title_match_re = False
    book_titles = set()
    book_titles_raw = set()

    # On recupère tout les titres des livres de la page sous forme de set
    for title, section in couple['author_wiki']['parsed_text'].items():
        try:
            if re.search(r'bibliographie|Œuvres|Œuvre|théâtre|poésie|nouvelles|essais|publications|romans|roman|Littérature|Ouvrages|Filmographie', title, re.IGNORECASE):
                book_titles.update(extract_title_from_list(section['list']))
                book_titles_raw.update(section['list'])
        except KeyError:
            pass

    # On vérifie si on peut trouver des titres proches en comparant les titres parsés
    # au titres normalisés de nos bases de donnée
    book_titles_DB = [book.title for book in couple['author_DB']['books']]
    for title in book_titles_DB:
        for title_wiki in book_titles:
            dist_titles = Levenshtein.distance(title, title_wiki)
            if dist_titles < max(1, min(len(title), len(title_wiki)) / 4):
                confirmed_by_title_match = True
                break
    # On recherche par le suite le titre du livre directement dans les titres non nettoyé de la page wikipédia
    # via expression régulières
    for title in book_titles_DB:
        for title_wiki_raw in book_titles_raw:
            if re.search(normalize(title), normalize(title_wiki_raw)):
                confirmed_by_title_match_re = True
                break

    # On liste les cas possibles: 0,1,2 tests réussis
    if (not confirmed_by_title_match) and (not confirmed_by_title_match_re):
        if book_titles and book_titles_DB:
            count_differentiate_by_titles += 1
            couples_infirmed.append(couple)
        else:
            count_no_titles += 1
            couples_unsure.append(couple)
    else:
        count_checked_by_titles += 1
        couples_confirmed.append(couple)

    # On sauvegarde comment avons nous trouvé les correspondances entre les deux listes de titres
    couple['confirmed_by_title_match'] = confirmed_by_title_match
    couple['confirmed_by_title_match_re'] = confirmed_by_title_match_re

print('Nombre de couple: ', len(couple_writers))
print('Nombre de couples confirmés par titres: ', count_checked_by_titles)
print('Nombre de couples infirmés par titres: ', count_differentiate_by_titles)
print('Nombre de couples sans titres de wikipedia: ', count_no_titles)

with open('../Data/Wikipedia/couple_authors_wikipedia_confirmed.json', 'w') as outfile:
    json.dump(couples_confirmed, outfile, cls=BookJSONEncoder)

with open('../Data/Wikipedia/couple_authors_wikipedia_infirmed.json', 'w') as outfile:
    json.dump(couples_infirmed, outfile, cls=BookJSONEncoder)

with open('../Data/Wikipedia/couple_authors_wikipedia_unsure.json', 'w') as outfile:
    json.dump(couples_unsure, outfile, cls=BookJSONEncoder)

On peut s'interésé a l'intersection des couples généré depuis wikidata

In [4]:
with open('../Data/Wikipedia/parsed_couple_writers.json', 'r') as couples_file:
    wikipedia_couple_writers = json.load(couples_file)

with open('../Data/Wikidata/couple_authors_wikidata.json', 'r') as couples_file:
    wikidata_couple_writers = json.load(couples_file)

count = 0
couples_wikipedia_wikidata = []
for wikipedia_couple in wikipedia_couple_writers:
    for wikidata_couple in wikidata_couple_writers:
        if wikidata_couple['author_DB']['name'] == wikipedia_couple['author_DB']['name']:
            count += 1
print("Nombre d'auteurs en communs: ", count)

with open('../Data/couple_authors_wikipedia_wikidata.json', 'w') as outfile:
    json.dump(couples_wikipedia_wikidata, outfile)



Nombre d'auteurs en communs:  765
